In [ ]:
!pip install lightly

In [2]:
!git clone https://github.com/alexeygrigorev/clothing-dataset-small.git

Cloning into 'clothing-dataset-small'...
remote: Enumerating objects: 3839, done.
remote: Counting objects: 100% (400/400), done.
remote: Compressing objects: 100% (400/400), done.
remote: Total 3839 (delta 9), reused 385 (delta 0), pack-reused 3439
Receiving objects: 100% (3839/3839), 100.58 MiB | 24.27 MiB/s, done.
Resolving deltas: 100% (10/10), done.


## Creation dataset on the Lightly Platform with embeddings


In [3]:
!lightly-train input_dir=clothing-dataset-small/train trainer.max_epochs=1

/usr/local/lib/python3.7/dist-packages/hydra/_internal/hydra.py:127: UserWarning: Future Hydra versions will no longer change working directory at job runtime by default.
See https://hydra.cc/docs/next/upgrades/1.1_to_1.2/changes_to_job_working_dir/ for more information.
  configure_logging=with_log_configuration,
/usr/local/lib/python3.7/dist-packages/lightly/cli/train_cli.py:69: UserWarning: Training a self-supervised model with a small batch size: 16! Small batch size may harm embedding quality. You can specify the batch size via the loader key-word: loader.batch_size=BSZ
  warnings.warn(msg)
Downloading: "https://storage.googleapis.com/models_boris/whattolabel-resnet18-simclr-d32-w1.0-i-085d0693.pth" to /root/.cache/torch/hub/checkpoints/whattolabel-resnet18-simclr-d32-w1.0-i-085d0693.pth
100% 42.8M/42.8M [00:00<00:00, 194MB/s]
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/core/lightning.py:23: LightningDeprecationWarning: pytorch_lightning.core.lightning.LightningModule

In [4]:
#Create embeddings for the dataset (replace **mycheckpoint.ckpt** with the model checkpoint from the lightly-train output), e.g. with

!lightly-embed input_dir="clothing-dataset-small/train" checkpoint="/content/lightly_outputs/2022-11-16/04-46-51/lightly_epoch_0.ckpt"

/usr/local/lib/python3.7/dist-packages/hydra/_internal/hydra.py:127: UserWarning: Future Hydra versions will no longer change working directory at job runtime by default.
See https://hydra.cc/docs/next/upgrades/1.1_to_1.2/changes_to_job_working_dir/ for more information.
  configure_logging=with_log_configuration,
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/core/lightning.py:23: LightningDeprecationWarning: pytorch_lightning.core.lightning.LightningModule has been deprecated in v1.7 and will be removed in v1.9. Use the equivalent class from the pytorch_lightning.core.module.LightningModule class instead.
  "pytorch_lightning.core.lightning.LightningModule has been deprecated in v1.7"
Compute efficiency: 0.15: 100% 3068/3068 [00:15<00:00, 193.20imgs/s]
Embeddings are stored at /content/lightly_outputs/2022-11-16/04-48-29/embeddings.csv


In [5]:
#Save the path to the **embeddings.csv**, you will need it later for uploading the embeddings and for defining the dataset for the classifier:
# rename the embeddings output to embeddings.csv
!mv '/content/lightly_outputs/2022-11-16/04-48-29/embeddings.csv' 'embeddings.csv'

In [6]:
#Create a new dataset on the lightly platform as described in `lightly-platform` and Save the **token** and **dataset id**, you will need them later to upload the images and embeddings and to run the active learning samplers.
!lightly-magic token='0496e4756268346002a999825678f0be22e29c8145447c50' dataset_id='63746bfff32e72da2b44a788' input_dir='/content/clothing-dataset-small/train' trainer.max_epochs=2

/usr/local/lib/python3.7/dist-packages/hydra/_internal/hydra.py:127: UserWarning: Future Hydra versions will no longer change working directory at job runtime by default.
See https://hydra.cc/docs/next/upgrades/1.1_to_1.2/changes_to_job_working_dir/ for more information.
  configure_logging=with_log_configuration,
########## Starting to train an embedding model.
/usr/local/lib/python3.7/dist-packages/lightly/cli/train_cli.py:69: UserWarning: Training a self-supervised model with a small batch size: 16! Small batch size may harm embedding quality. You can specify the batch size via the loader key-word: loader.batch_size=BSZ
  warnings.warn(msg)
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/core/lightning.py:23: LightningDeprecationWarning: pytorch_lightning.core.lightning.LightningModule has been deprecated in v1.7 and will be removed in v1.9. Use the equivalent class from the pytorch_lightning.core.module.LightningModule class instead.
  "pytorch_lightning.core.lightning.Lig

## Active learning
Import the Python frameworks we need for this tutorial.



In [7]:
import os
import csv
from typing import List, Dict, Tuple
import numpy as np
from sklearn.neighbors import KNeighborsClassifier

from lightly.active_learning.agents.agent import ActiveLearningAgent
# from lightly.active_learning.config.sampler_config import SamplerConfig
from lightly.active_learning.scorers.classification import ScorerClassification
from lightly.api.api_workflow_client import ApiWorkflowClient
from lightly.openapi_generated.swagger_client import SamplingMethod

Define the parameters (make sure to set the path to your embeddings file).



In [8]:
path_to_embeddings_csv = "/content/embeddings.csv"
YOUR_TOKEN = "0496e4756268346002a999825678f0be22e29c8145447c50"  # your token of the web platform
YOUR_DATASET_ID = "63746bfff32e72da2b44a788"  # the id of your dataset on the web platform

In [16]:
class CSVEmbeddingDataset:
    def __init__(self, path_to_embeddings_csv: str):
        with open(path_to_embeddings_csv, 'r') as f:
            data = csv.reader(f)

            rows = list(data)
            header_row = rows[0]
            rows_without_header = rows[1:]

            index_filenames = header_row.index('filenames')
            filenames = [row[index_filenames] for row in rows_without_header]

            index_labels = header_row.index('labels')
            labels = [row[index_labels] for row in rows_without_header]

            embeddings = rows_without_header
            indexes_to_delete = sorted([index_filenames, index_labels], reverse=True)
            for embedding_row in embeddings:
                for index_to_delete in indexes_to_delete:
                    del embedding_row[index_to_delete]

        # create the dataset as a dictionary mapping from the filename to a tuple of the embedding and the label
        self.dataset: Dict[str, Tuple[np.ndarray, int]] = \
            dict([(filename, (np.array(embedding_row, dtype=float), int(label)))
                  for filename, embedding_row, label in zip(filenames, embeddings, labels)])

    def get_features(self, filenames: List[str]) -> np.ndarray:
        features_array = np.array([self.dataset[filename][0] for filename in filenames])
        return features_array

    def get_labels(self, filenames: List[str]) -> np.ndarray:
        labels = np.array([self.dataset[filename][1] for filename in filenames])
        return labels

In [17]:
!export LIGHTLY_SERVER_LOCATION="https://api-dev.lightly.ai"

In [30]:
os.environ['LIGHTLY_SERVER_LOCATION']="https://api-dev.lightly.ai"
api_workflow_client = ApiWorkflowClient(token="d306112ae626405ea7238f22", dataset_id="60f142c8fb1e4200276d6ebf")
api_workflow_client.upload_embeddings(name="embedding-1", path_to_embeddings_csv=path_to_embeddings_csv)

Appending embeddings from server.


Define the dataset for the classifer, the classifier and the active learning agent



In [ ]:
# api_workflow_client.upload_embeddings(name="embedding-1", path_to_embeddings_csv=path_to_embeddings_csv)

In [31]:
dataset = CSVEmbeddingDataset(path_to_embeddings_csv=path_to_embeddings_csv)
classifier = KNeighborsClassifier(n_neighbors=20, weights='distance')

In [32]:
agent = ActiveLearningAgent(api_workflow_client=api_workflow_client)

1. Choose an initial subset of your dataset.
We want to start with 100 samples and use the CORESET sampler for sampling them.



In [33]:
import warnings
from datetime import datetime

from lightly.openapi_generated.swagger_client.models.sampling_method import SamplingMethod

In [34]:
class SelectionConfig:
    def __init__(self, method: SamplingMethod = SamplingMethod.CORESET, n_samples: int = 32, min_distance: float = -1,
                 name: str = None):

        self.method = method
        self.n_samples = n_samples
        self.min_distance = min_distance
        if name is None:
            date_time = datetime.now().strftime("%m_%d_%Y__%H_%M_%S")
            name = f"{self.method}_{self.n_samples}_{self.min_distance}_{date_time}"
        self.name = name


class SamplingConfig(SelectionConfig):

    def __init__(self, *args, **kwargs):
        warnings.warn(PendingDeprecationWarning(
            "SamplingConfig() is deprecated "
            "in favour of SelectionConfig() "
            "and will be removed in the future."
        ), )
        SelectionConfig.__init__(self, *args, **kwargs)


In [36]:
config = SelectionConfig(
  n_samples=100, 
  method=SamplingMethod.CORAL, 
  name='active-learning-loop-12'
)
agent.query(selection_config=config)

2. Train a classifier on the labeled set.



In [37]:
labeled_set_features = dataset.get_features(agent.labeled_set)
labeled_set_labels = dataset.get_labels(agent.labeled_set)
classifier.fit(X=labeled_set_features, y=labeled_set_labels)

KNeighborsClassifier(n_neighbors=20, weights='distance')

3. Use the classifier to predict on the query set.



In [38]:
query_set_features = dataset.get_features(agent.query_set)
predictions = classifier.predict_proba(X=query_set_features)

4. Calculate active learning scores from the prediction.



In [39]:
active_learning_scorer = ScorerClassification(model_output=predictions)

5. Use an active learning agent to choose the next samples to be labeled based on the active learning scores.
We want to sample another 100 samples to have 200 samples in total and use the active learning sampler CORAL for it.



In [41]:
selection_config = SelectionConfig(n_samples=200, method=SamplingMethod.CORAL, name='al-iteration-23')
agent.query(selection_config=selection_config, al_scorer=active_learning_scorer)
print(f"There are {len(agent.labeled_set)} samples in the labeled set.")

There are 200 samples in the labeled set.


6. Update the labeled set to include the newly chosen samples and remove them from the unlabeled set.
This is already done internally inside the ActiveLearningAgent - no work for you :)



Now you can use the newly chosen labeled set to retrain you classifer on it.
You can evaluate it e.g. on the unlabeled set, or on embeddings of a test set you generated before.
If you are not satisfied with the performance, you can run steps 2 to 5 again.



In [42]:
labeled_set_features = dataset.get_features(agent.labeled_set)
labeled_set_labels = dataset.get_labels(agent.labeled_set)
classifier.fit(X=labeled_set_features, y=labeled_set_labels)

# evaluate on unlabeled set
unlabeled_set_features = dataset.get_features(agent.unlabeled_set)
unlabeled_set_labels = dataset.get_labels(agent.unlabeled_set)
accuracy_on_unlabeled_set = classifier.score(X=unlabeled_set_features, y=unlabeled_set_labels)
print(f"accuracy on unlabeled set: {accuracy_on_unlabeled_set}")

accuracy on unlabeled set: 0.35704323570432356
